In [1]:
import os
from pathlib import Path
path = Path(os.getcwd())
os.chdir(path.parent.absolute())

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import pandas_ta as ta
import vectorbt as vbt
import mt4_hst

# Create your own Custom Strategy
TrendStrategy = ta.Strategy(
    name="Trend Signal Strategy",
    description="SMA 50,200, BBANDS, RSI, MACD and Volume SMA 20",
    ta=[
        {"kind": "percent_return", "length": 1},
        {"kind": "sma", "length": 50},
        {"kind": "ema", "length": 10},
        {"kind": "ema", "length": 20},
        {"kind": "donchian", "lower_length": 10, "upper_length": 10},
        {"kind": "donchian", "lower_length": 20, "upper_length": 20},
        {"kind": "donchian", "lower_length": 50, "upper_length": 50},
        {"kind": "sma", "close": "volume", "length": 20, "prefix": "VOLUME"},
    ]
)

In [2]:
cheight, cwidth = 500, 1000 # Adjust as needed for Chart Height and Width
vbt.settings.set_theme("seaborn") # Options: "light" (Default), "dark" (my fav), "seaborn"

# Must be set
vbt.settings.portfolio["freq"] = "1D" # Daily

# Predefine vectorbt Portfolio settings
vbt.settings.portfolio["init_cash"] = 1e4
vbt.settings.portfolio["fees"] = 0.0015 # 0.25%
vbt.settings.portfolio["slippage"] = 0.0015 # 0.25%
vbt.settings.portfolio["size"] = 100
# vbt.settings.portfolio["accumulate"] = False
vbt.settings.portfolio["allow_partial"] = False

In [3]:
# v1
# Example Long Trends for the selected Asset
# * Uncomment others for exploration or replace them with your own TA Trend Strategy
def trends(df: pd.DataFrame):
    # create neccesary indicators
    df.ta.percent_return(length=1, append=True)
    df.ta.sma(50, append=True)
    df.ta.ema(10, append=True)
    df.ta.ema(20, append=True)
    df.ta.donchian(lower_length=20, upper_length=20, append=True)
    df.ta.donchian(lower_length=10, upper_length=10, append=True)
    df.ta.donchian(lower_length=50, upper_length=50, append=True)
    df.ta.sma(close=df['volume'], length=20, prefix="VOLUME", append=True)
    
    # condition
    trend_cond = (
        (df['close'] > df['SMA_50'])
        & (df['EMA_10'] > df['EMA_20'])
        & (df['DCL_20_20'] > df['DCL_50_50'])
        # & (df['close'] > df['EMA_20'])
    ).astype(int)
    
    return trend_cond
    # return ta.ma(mamode, df.close, length=fast) > ta.ma(mamode, df.close, length=slow) # SMA(fast) > SMA(slow) "Golden/Death Cross"
#     return ta.increasing(ta.ma(mamode, df.close, length=fast)) # Increasing MA(fast)
#     return ta.macd(df.close, fast, slow).iloc[:,1] > 0 # MACD Histogram is positive

def trends_confirm_entries(df: pd.DataFrame):
    # candle pattern
    cdl_pattern = df.ta.cdl_pattern(
        name=["closingmarubozu", "marubozu", "engulfing", "longline"], 
        scalar=1).sum(axis=1).astype(bool).astype(int)
    volume_breakout = (df['volume'] >= 1.25 * ta.sma(df['volume'], 20)).astype(int)
    df['breakout'] = (
        cdl_pattern 
        * volume_breakout 
        # * (df['PCTRET_1'] > 0.04) 
        * (df['close'] > df['DCU_10_10'].shift())
    )
    
    # modify trends
    def modify_trend(df):
        return (
            df['trends'] * df['breakout']
        ).replace(0, np.nan).ffill().fillna(0).astype(int)

    groups = (df.trends != df.trends.shift()).cumsum()
    # df['groups'] = groups
    df['trends'] = df.groupby(groups).apply(modify_trend).to_list()
    df['trends'] = df['trends'].astype(bool)
    return df

In [33]:
# v2
def trends(df: pd.DataFrame):
    # create neccesary indicators
    df.ta.strategy(TrendStrategy)
    
    # condition
    trend_cond = (
        (df['close'] > df['SMA_50'])
        & (df['EMA_10'] > df['EMA_20'])
        & (df['DCL_20_20'] > df['DCL_50_50'])
    ).astype(int)
    
    return trend_cond

def trends_confirm_entries(df: pd.DataFrame):
    volume_breakout = (df['volume'] >= 1.25 * df['VOLUME_SMA_20']).astype(int)
    
    # candle pattern
    entry_pattern = df.ta.cdl_pattern(
        name=["closingmarubozu", "marubozu", "engulfing", "longline"], 
        scalar=1).sum(axis=1).astype(int)
    
    exit_pattern = df.ta.cdl_pattern(
        name=[
            "doji", "dojistar", "dragonflydoji", "eveningdojistar", "invertedhammer",
            "eveningstar", "gravestonedoji", "hangingman", "marubozu", "3blackcrows",
            "longleggeddoji", "shootingstar", "spinningtop"
        ], 
        scalar=1).sum(axis=1).astype(int)
    
    df['entry'] = ((entry_pattern > 0) * volume_breakout * (df['close'] > df['DCU_10_10'].shift())).astype(bool).astype(int)
    df['exit'] = ((exit_pattern < 0) * volume_breakout).astype(bool).astype(int)
    
    def modify_entry_exit(df):
        if (df['entry'].sum() > 0):
            if (df['exit'].sum() == 0):
                trends = (df['trends'] * df['entry']).replace(0, np.nan).ffill()
            else:
                trends = (df['trends'] * (df['entry'] + df['exit'])).replace(0, np.nan).interpolate(limit_area='inside', method='nearest')
        else:
            trends = df['trends'] * df['entry']
        return trends.fillna(0).astype(int).astype(bool)

    df['trends'] = df.groupby((df.trends != df.trends.shift()).cumsum()).apply(modify_entry_exit).to_list()
    return df

In [34]:
ticker = 'HAX'
benchmark = 'VNINDEX'
# df = mt4_hst.read_hst("stock_env/datasets/FPT1440.hst")

benchmark = mt4_hst.read_hst("../stock_datasets/" + benchmark + "1440.hst")
benchmark = benchmark[['time', 'close']].rename(columns={'close': 'benchmark'})
df = mt4_hst.read_hst("../stock_datasets/" + ticker + "1440.hst")
df = df.merge(benchmark, how='inner', on='time')

df = df.sort_values('time')
df.index = df['time']
df['trends'] = trends(df)
df = trends_confirm_entries(df)
df.ta.tsignals(df['trends'], asbool=True, append=True);
# df.to_csv('temp/signal/' + ticker + '.csv', index=False)

In [35]:
df.loc['2022-07-22':'2022-08-03']

,time,open,high,low,close,volume,benchmark,PCTRET_1,SMA_50,EMA_10,...,DCM_50_50,DCU_50_50,VOLUME_SMA_20,trends,entry,exit,TS_Trends,TS_Trades,TS_Entries,TS_Exits
time,,,,,,,,,,,,,,,,,,,,,
2022-07-22,2022-07-22,19.000000,19.600000,18.799999,18.850000,709600.0,1194.760010,0.002660,17.492329,18.314065,...,17.079214,20.393091,520740.0,False,0,0,False,0,False,False
2022-07-25,2022-07-25,18.850000,20.150000,18.799999,20.150000,1557100.0,1188.500000,0.068965,17.592831,18.647871,...,17.079214,20.393091,588495.0,True,1,0,True,1,True,False
2022-07-26,2022-07-26,20.600000,21.200001,20.500000,20.549999,874400.0,1185.069946,0.019851,17.680941,18.993713,...,17.482668,21.200001,610220.0,True,0,0,True,0,False,False
2022-07-27,2022-07-27,20.850000,21.600000,20.100000,21.450001,727700.0,1191.040039,0.043796,17.788750,19.440311,...,17.682668,21.600000,629655.0,True,0,0,True,0,False,False
2022-07-28,2022-07-28,22.000000,22.000000,21.100000,21.150000,1060000.0,1208.119995,-0.013986,17.890558,19.751163,...,17.882668,22.000000,670415.0,True,0,1,True,0,False,False
2022-07-29,2022-07-29,21.299999,21.850000,20.700001,21.150000,776600.0,1206.329956,0.000000,17.970275,20.005497,...,17.882668,22.000000,695710.0,True,0,0,True,0,False,False
2022-08-01,2022-08-01,21.500000,21.850000,20.799999,21.100000,882100.0,1231.349976,-0.002364,18.037945,20.204498,...,17.882668,22.000000,730240.0,True,0,0,True,0,False,False
2022-08-02,2022-08-02,20.950001,22.500000,20.650000,22.500000,1442800.0,1241.619995,0.066351,18.137014,20.621862,...,18.132668,22.500000,791430.0,True,1,0,True,0,False,False
2022-08-03,2022-08-03,22.000000,22.500000,21.750000,22.150000,947300.0,1249.760010,-0.015556,18.206990,20.899705,...,18.132668,22.500000,829845.0,False,0,0,False,-1,False,True


In [15]:
df = mt4_hst.read_hst("../stock_datasets/HAX1440.hst")
df.index = df['time']
candle_pattern = df.ta.cdl_pattern(
        name=[
            "doji", "dojistar", "dragonflydoji", "eveningdojistar", "invertedhammer",
            "eveningstar", "gravestonedoji", "hangingman", "marubozu", "3blackcrows",
            "longleggeddoji", "shootingstar", "spinningtop"
        ], 
        scalar=1).sum(axis=1)
df['candle_pattern'] = (candle_pattern<0)
# df[df['candle_pattern'] == True].tail()
df.to_csv('temp/candle_pattern/HAX.csv', index=False)

In [16]:
df = mt4_hst.read_hst("../stock_datasets/HAX1440.hst")
df.index = df['time']
candle_pattern = df.ta.cdl_pattern(
        name=[
            "doji", "dojistar", "dragonflydoji", "eveningdojistar", "invertedhammer",
            "eveningstar", "gravestonedoji", "hangingman", "marubozu", "3blackcrows",
            "longleggeddoji", "shootingstar", "spinningtop"
        ], 
        scalar=1)
candle_pattern.loc['2022-08-03']

CDL_DOJI_10_0.1        0.0
CDL_DOJISTAR           0.0
CDL_DRAGONFLYDOJI      0.0
CDL_EVENINGDOJISTAR    0.0
CDL_INVERTEDHAMMER     0.0
CDL_EVENINGSTAR        0.0
CDL_GRAVESTONEDOJI     0.0
CDL_HANGINGMAN         0.0
CDL_MARUBOZU           0.0
CDL_3BLACKCROWS        0.0
CDL_LONGLEGGEDDOJI     0.0
CDL_SHOOTINGSTAR       0.0
CDL_SPINNINGTOP        1.0
Name: 2022-08-03 00:00:00, dtype: float64

# Backtest

In [31]:
from stock_env.utils import create_performance

# Asset Portfolio from Trade Signals
assetpf_signals = vbt.Portfolio.from_signals(
    df.close,
    entries=df.TS_Entries,
    exits=df.TS_Exits,
)
assetpf_signals.stats(settings=dict(benchmark_rets=df.benchmark.pct_change()))

Start                                2006-12-26 00:00:00
End                                  2022-08-10 00:00:00
Period                                3890 days 00:00:00
Start Value                                      10000.0
End Value                                   11209.130963
Total Return [%]                                12.09131
Benchmark Return [%]                           67.906716
Max Gross Exposure [%]                         28.838022
Total Fees Paid                                71.303935
Max Drawdown [%]                                6.736862
Max Drawdown Duration                 1587 days 00:00:00
Total Trades                                          27
Total Closed Trades                                   26
Total Open Trades                                      1
Open Trade PnL                                198.950581
Win Rate [%]                                   42.307692
Best Trade [%]                                 66.585503
Worst Trade [%]                

In [ ]:
values = assetpf_signals.value()
values.index = df['time']
returns = values.pct_change()
create_performance(returns)

In [6]:
fig = df.close.vbt.plot(trace_kwargs=dict(name='Close'), autosize=False,width=1000,height=500)
assetpf_signals.positions.plot(close_trace_kwargs=dict(visible=False), fig=fig)

FigureWidget({
    'data': [{'name': 'Close',
              'showlegend': True,
              'type': 'scatter…

In [ ]:
values.vbt.drawdowns.plot(top_n=3)

In [32]:
fig = df.close.vbt.plot(trace_kwargs=dict(name='Close'), autosize=False,width=1000,height=500)
assetpf_signals.positions.plot(close_trace_kwargs=dict(visible=False), fig=fig)

FigureWidget({
    'data': [{'name': 'Close',
              'showlegend': True,
              'type': 'scatter…